In [ ]:
import pandas as pd
import numpy as np

In [ ]:
!pip install stellargraph

In [ ]:
!pip install chardet

In [ ]:
import stellargraph as sg
from stellargraph.mapper import PaddedGraphGenerator
from stellargraph.layer import GCNSupervisedGraphClassification
from stellargraph import StellarGraph

In [ ]:
from sklearn import model_selection
from sklearn.metrics import confusion_matrix,auc,precision_recall_curve,roc_curve
from tensorflow.keras import Model
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import binary_crossentropy,mean_squared_error
from tensorflow.keras.callbacks import EarlyStopping
import tensorflow as tf
from stellargraph import IndexedArray

In [ ]:
import itertools
import matplotlib.pyplot as plt

In [ ]:
def read_para(path):
    df = pd.read_csv(path,sep='\t',header=None)
    dic = {}
    for i in range(df.shape[0]):
        hla = df[0].iloc[i]
        paratope = df[1].iloc[i]
        try:
            dic[hla] = paratope
        except KeyError:
            dic[hla] = []
            dic[hla].append(paratope)
    return dic

In [ ]:
def dict_inventory(inventory):
    dicA, dicB, dicC = {}, {}, {}
    dic = {'A': dicA, 'B': dicB, 'C': dicC}

    for hla in inventory:
        type_ = hla[4]  # A,B,C
        first2 = hla[6:8]  # 01
        last2 = hla[8:]  # 01
        try:
            dic[type_][first2].append(last2)
        except KeyError:
            dic[type_][first2] = []
            dic[type_][first2].append(last2)

    return dic

In [ ]:
def rescue_unknown_hla(hla, dic_inventory):
    type_ = hla[4]
    first2 = hla[6:8]
    last2 = hla[8:]
    big_category = dic_inventory[type_]
    #print(hla)
    if not big_category.get(first2) == None:
        small_category = big_category.get(first2)
        distance = [abs(int(last2) - int(i)) for i in small_category]
        optimal = min(zip(small_category, distance), key=lambda x: x[1])[0]
        return 'HLA-' + str(type_) + '*' + str(first2) + str(optimal)
    else:
        small_category = list(big_category.keys())
        distance = [abs(int(first2) - int(i)) for i in small_category]
        optimal = min(zip(small_category, distance), key=lambda x: x[1])[0]
        return 'HLA-' + str(type_) + '*' + str(optimal) + str(big_category[optimal][0])

In [ ]:
class Graph_Constructor():

    @staticmethod
    def combinator(pep,hla):
        source = ['p' + str(i+1) for i in range(len(pep))]
        target = ['h' + str(i+1) for i in range(len(hla))]
        return source,target

    @staticmethod
    def numerical(pep,hla,after_pca,embed=12):   # after_pca [21,12]
        pep = pep.replace('X','-').upper()
        hla = hla.replace('X','-').upper()
        feature_array_pep = np.empty([len(pep),embed])
        feature_array_hla = np.empty([len(hla),embed])
        amino = 'ARNDCQEGHILKMFPSTWYV-'
        for i in range(len(pep)):
            feature_array_pep[i,:] = after_pca[amino.index(pep[i]),:]
        for i in range(len(hla)):
            feature_array_hla[i,:] = after_pca[amino.index(hla[i]),:]
        feature_array = np.concatenate([feature_array_pep,feature_array_hla],axis=0)
        #print(feature_array_pep.shape,feature_array_hla.shape,feature_array.shape)
        return feature_array



    # define the function to convert the weight column
    @staticmethod
    def DHT_numerical(weight, feature_array):
        if weight[0] == 'p':
            idx = int(weight[1:]) - 1
            return feature_array[idx]
        elif weight[0] == 'h':
            idx = int(weight[1:]) + 8
            return feature_array[idx]
        else:
            return None

    # define the function to Split the features values(orginal node features)
    @staticmethod
    def Split_Weight(row):
        #return pd.Series(row['weight'].strip('[]').split())
        return pd.Series(row['weight'].tolist())



    @staticmethod
    def DHT_numerical_notused(Edges_df,feature_array):   # after_pca [21,12]
        amino = 'ARNDCQEGHILKMFPSTWYV-'

        print("type of feature_array[1]:",type(feature_array[1]))

        for i in range(len(Edges_df)):
            weight_col = Edges_df.iloc[i]['weight']
            first_letter = weight_col[0]
            number = re.findall('\d+', weight_col)[0]
            #print("in DHT Numerical edges weight is : ",i,weight_col)
            #print("in DHT Numerical edges weight1: ",i,first_letter)
            #print("in DHT Numerical edges weight2: ",i,number)
            #print("type of number:",type(number))
            #print("type of first_letter:",type(first_letter))


            if (first_letter =='p'):
                print("In p")
                Edges_df.at[i,'weight'] = 1 #feature_array[1]

            elif (first_letter =='h'):
                print("In h")
                Edges_df.at[i,'weight'] = 2 #feature_array[1+9]

        #Edges_df.to_csv('D:\\PhD\\GraphImmuno_NewResults\\data\\DHT_numerical.txt', sep='\t', index=False)
        return None


    @staticmethod
    def unweight_edge(pep,hla,after_pca):
        source,target = Graph_Constructor.combinator(pep,hla)
        combine = list(itertools.product(source,target))
        weight = itertools.repeat(1,len(source)*len(target))
        edges = pd.DataFrame({'source':[item[0] for item in combine],'target':[item[1] for item in combine],'weight':weight})
        feature_array = Graph_Constructor.numerical(pep,hla,after_pca)
        try:nodes = IndexedArray(feature_array,index=source+target)
        except: print(pep,hla,feature_array.shape)
        graph = StellarGraph(nodes,edges,node_type_default='corner',edge_type_default='line')
        return graph


    @staticmethod
    def Update_Edges_new(pep,hla,edges):

      pep = pep.replace('X','-').upper()
      hla = hla.replace('X','-').upper()

      amino = 'ARNDCQEGHILKMFPSTWYV-'

      edges_weights = pd.read_csv(r'D:\PhD\New_Graph_Model\data\Edge_weights_pca_array_new.txt',sep=' ',header = None)

      # create a dictionary to map amino acid characters to their position in the amino acid string
      amino_dict = {}
      for i, c in enumerate(amino):
        amino_dict[c] = i

      # loop through the rows of the edges dataframe
      for index, row in edges.iterrows():
            # get the source and target nodes from the row
            source = row['source']
            target = row['target']


            # check if source is a peptide or HLA allele
            if source[0] == 'p':
            # get the position of the amino acid in the peptide sequence
                pos = int(source[1:]) - 1

                # get the amino acid character at that position
                amino_char = pep[pos]

                # get the index of the amino acid character in the amino acid string
                amino_pos = amino_dict[amino_char]

            elif source[0] == 'h':
                # get the position of the amino acid in the HLA allele
                pos = int(source[1:]) - 1

                # get the amino acid character at that position
                amino_char = hla[pos]

                # get the index of the amino acid character in the amino acid string
                amino_pos = amino_dict[amino_char]

            # check if target is a peptide or HLA allele
            if target[0] == 'p':
                # get the position of the amino acid in the peptide sequence
                pos = int(target[1:]) - 1

                # get the amino acid character at that position
                amino_char = pep[pos]

                # get the index of the amino acid character in the amino acid string
                amino_pos2 = amino_dict[amino_char]

            elif target[0] == 'h':
                # get the position of the amino acid in the HLA allele
                pos = int(target[1:]) - 1

                # get the amino acid character at that position
                amino_char = hla[pos]

                # get the index of the amino acid character in the amino acid string
                amino_pos2 = amino_dict[amino_char]

                # get the weight value from the weights dataframe
                weight = edges_weights.iloc[amino_pos, amino_pos2]

            # update the weight column in the edges dataframe
            #edges.at[index, 'weight'] = weight
            edges['weight'] = edges['weight'].astype(float)
            edges.at[index, 'weight'] = (weight)

      # Check if source[0] and target[0] are same, if not update weight accordingly
      for index, row in edges.iterrows():
        source, target = row['source'], row['target']
        weight = row['weight']
        mean = np.mean(edges['weight'])
        std = np.std(edges['weight'])

        if source[0] != target[0] and (weight > (mean + std) or weight < (mean - std)):
            edges.at[index, 'weight'] = -1

      return edges


    @staticmethod
    def Update_Edges(pep,hla,edges):
      #print("edges",edges)

      #pep = "KEHVFFSEY"
      #hla = "MMMMYYTMMKKRKYREISNT-ENNTT--YIYDYTKWVVQDYLSRY-"

      pep = pep.replace('X','-').upper()
      hla = hla.replace('X','-').upper()

      print("pep is :",pep)
      print("hla is :",hla)
      print("edges",edges)
      edges.to_csv('D:\\PhD\\New_Graph_Model\\orginal_edges.txt', sep='\t', index=False)


      #edges_weights = pd.read_csv('Edge_Weights_V3.txt',sep='\t',header = None)
      #edges_weights = pd.read_csv('D:\\PhD\\GraphImmuno_NewResults\\data\\Edge_Weights_V3.txt',sep='\t',header = None)
      edges_weights = pd.read_csv(r'D:\PhD\New_Graph_Model\data\Edge_weights_pca_array_new.txt',sep=' ',header = None)
      print("edges_weights size",edges_weights.shape)
      print("edges_weights",edges_weights)

      amino = 'ARNDCQEGHILKMFPSTWYV-'

      # add a new row and column with values zero
      #edges_weights = np.pad(edges_weights, [(0,1),(0,1)], mode='constant')

      # calculate the mean and standard deviation
      mean = np.mean(edges_weights)
      std = np.std(edges_weights)

      edges['weight'] = edges['weight'].astype(float)

      for i in range(0, len(edges)):

        if ((edges.iloc[i]['source'][0] == "p") and (edges.iloc[i]['target'][0] == "p")):
          edges.at[i,'weight'] = edges_weights[amino.index(pep[(int([edges.iloc[i]['source'][1:]][0])-1)])][amino.index(pep[(int([edges.iloc[i]['target'][1:]][0])-1)])]

        elif ((edges.iloc[i]['source'][0] == "p") and (edges.iloc[i]['target'][0] == "h")):
          edges.at[i,'weight'] = edges_weights[amino.index(pep[(int([edges.iloc[i]['source'][1:]][0])-1)])][amino.index(hla[(int([edges.iloc[i]['target'][1:]][0])-1)])]
          if ((edges.at[i,'weight'] > (mean + std)) or (edges.at[i,'weight'] < ((mean - std)))):
            edges.at[i,'weight'] = -1

        elif ((edges.iloc[i]['source'][0] == "h") and (edges.iloc[i]['target'][0] == "p")):
          edges.at[i,'weight'] = edges_weights[amino.index(hla[(int([edges.iloc[i]['source'][1:]][0])-1)])][amino.index(pep[(int([edges.iloc[i]['target'][1:]][0])-1)])]
          if ((edges.at[i,'weight'] > (mean + std)) or (edges.at[i,'weight'] < ((mean - std)))):
            edges.at[i,'weight'] = -1

        elif ((edges.iloc[i]['source'][0] == "h") and (edges.iloc[i]['target'][0] == "h")):
          edges.at[i,'weight'] = edges_weights[amino.index(hla[(int([edges.iloc[i]['source'][1:]][0])-1)])][amino.index(hla[(int([edges.iloc[i]['target'][1:]][0])-1)])]

      edges['weight'] = edges['weight'].astype(float)

      return edges


    #Dual Hyper transformation
    @staticmethod
    def DHT_new(source,target,edges,feature_array):

      new_nodeslist= edges.index.tolist()
      New_weights = edges["weight"].tolist()



      Edges_df = pd.DataFrame(columns=['source', 'target', 'weight'])

      # Reset the index to a named column
      edges = edges.rename_axis('index').reset_index()


      Edges_df = pd.DataFrame(columns=['source', 'target', 'weight'])


      # Updating the nodes edges by refering to edges_dataframe
      for i in range(len(edges)):
            for j in range(i+1, len(edges)):
                if edges.loc[i, 'source'] == edges.loc[j, 'source']:
                    weight = edges.loc[i, 'source']
                    Edges_df.loc[len(Edges_df)] = [edges.loc[i, 'index'], edges.loc[j, 'index'], weight]
                elif edges.loc[i, 'source'] == edges.loc[j, 'target']:
                    weight = edges.loc[i, 'source']
                    Edges_df.loc[len(Edges_df)] = [edges.loc[i, 'index'], edges.loc[j, 'index'], weight]
                elif edges.loc[i, 'target'] == edges.loc[j, 'source']:
                    weight = edges.loc[i, 'target']
                    Edges_df.loc[len(Edges_df)] = [edges.loc[i, 'index'], edges.loc[j, 'index'], weight]
                elif edges.loc[i, 'target'] == edges.loc[j, 'target']:
                    weight = edges.loc[i, 'target']
                    Edges_df.loc[len(Edges_df)] = [edges.loc[i, 'index'], edges.loc[j, 'index'], weight]


      #Edges_df.to_csv('D:\\PhD\\New_Graph_Model\\edges_df_values.txt', sep='\t', index=False)

      # replace empty values with NaN
      Edges_df = Edges_df.replace(r'^\s*$', np.nan, regex=True)

      # drop rows where 'weight' column has null or empty values
      Edges_df = Edges_df.dropna(subset=['weight'])


      new_nodeslist= pd.concat([Edges_df['source'], Edges_df['target']]).unique().tolist()
      New_nodes = pd.DataFrame(    {"x": New_weights}, index=new_nodeslist)


      # apply the function to the weight column
      Edges_df['weight'] = Edges_df['weight'].apply(Graph_Constructor.DHT_numerical, feature_array=feature_array)


      # Apply the function to the "weight" column to get a new DataFrame
      new_df = Edges_df.apply(Graph_Constructor.Split_Weight, axis=1)

      # Rename the columns in the new DataFrame
      new_df.columns = ['weight_'+str(i) for i in range(new_df.shape[1])]

      # Concatenate the new DataFrame with the original DataFrame
      Edges_df = pd.concat([Edges_df, new_df], axis=1)

      # Drop the original "weight" column
      Edges_df.drop('weight', axis=1, inplace=True)

      Edges_df.to_csv('D:\\PhD\\GraphImmuno_NewResults\\data\\Edges_df_values_New.txt', sep='\t', index=False)

      return New_nodes,Edges_df



    #Dual Hyper transformation
    @staticmethod
    def DHT(source,target,edges,feature_array):

      new_nodeslist= edges.index.tolist()
      New_weights = edges["weight"].tolist()

      #print("new_nodes in DHT", new_nodeslist)
      #print("New_weights", New_weights)
      #print("Existing edges : ", edges)

      #feature_weights = np.array(New_weights)
      #feature_weights_new = np.zeros((feature_weights.shape[0], 2))

      #for i in range(feature_weights_new.shape[0]):
        #feature_weights_new[i:, 0] = New_weights[i]


      #print("feature_weights_new",feature_weights_new)
      #print('feature_array Type',np.shape(feature_weights_new))
      #print('index Type',np.shape(new_nodeslist))


      # Finding the pairs
      new_substuff = []
      for L in range(1):
        for subset in itertools.combinations(new_nodeslist, 2):
          if len(subset) == 2:
            new_substuff.append(subset)

      # Converting the pairs to data frames
      Edges_df = pd.DataFrame(new_substuff)
      Edges_df.columns = ["Source", "Target"]
      Edges_df['Edge'] = ''

      #print(Edges_df.tail(5))
      #print(len(Edges_df))

      # Reset the index to a named column
      edges = edges.rename_axis('index').reset_index()

      # export dataframe to a text file
      #edges.to_csv('D:\\PhD\\GraphImmuno_NewResults\\data\\Existing_edges.txt', sep='\t', index=False)

      # Updating the edges by referingto edges dataframe
      for i in range(len(Edges_df)):


        if((edges.loc[edges['index'] == Edges_df.iloc[i]['Source']]['source']).values == (edges.loc[edges['index'] == Edges_df.iloc[i]['Target']]['source']).values):
          Edges_df.at[i,'Edge'] = edges.loc[edges['index'] == Edges_df.iloc[i]['Source']]['source'].values[0]

        if((edges.loc[edges['index'] == Edges_df.iloc[i]['Source']]['target']).values == (edges.loc[edges['index'] == Edges_df.iloc[i]['Target']]['target']).values):
          Edges_df.at[i,'Edge'] = edges.loc[edges['index'] == Edges_df.iloc[i]['Source']]['target'].values[0]

        if((edges.loc[edges['index'] == Edges_df.iloc[i]['Source']]['source']).values == (edges.loc[edges['index'] == Edges_df.iloc[i]['Target']]['target']).values):
          Edges_df.at[i,'Edge'] = edges.loc[edges['index'] == Edges_df.iloc[i]['Source']]['source'].values[0]

        if((edges.loc[edges['index'] == Edges_df.iloc[i]['Source']]['target']).values == (edges.loc[edges['index'] == Edges_df.iloc[i]['Target']]['source']).values):
          Edges_df.at[i,'Edge'] = edges.loc[edges['index'] == Edges_df.iloc[i]['Source']]['target'].values[0]


      Edges_df.rename(columns={'Source': 'source', 'Target': 'target', 'Edge': 'weight'}, inplace=True)

      #print('Edges_df Type1',(Edges_df))
      #Edges_df.to_csv('D:\\PhD\\GraphImmuno_NewResults\\data\\Edges_df_before.txt', sep='\t', index=False)


      # replace empty values with NaN
      Edges_df = Edges_df.replace(r'^\s*$', np.nan, regex=True)

      # drop rows where 'weight' column has null or empty values
      Edges_df = Edges_df.dropna(subset=['weight'])


      #print("Edges_df",Edges_df)
      #Edges_df.to_csv('D:\\PhD\\GraphImmuno_NewResults\\data\\Edges_df_After.txt', sep='\t', index=False)

      new_nodeslist= pd.concat([Edges_df['source'], Edges_df['target']]).unique().tolist()
      #New_nodes = IndexedArray(feature_weights_new, index=new_nodeslist)

      # Creating a the node df using the nodes ;ist as index and weihts as column.
      #feature_weights_new_list = [sublst[0] for sublst in feature_weights_new.tolist()]
      #New_nodes_df = pd.DataFrame(    {"x": feature_weights_new_list}, index=new_nodeslist)
      New_nodes = pd.DataFrame(    {"x": New_weights}, index=new_nodeslist)


      # apply the function to the weight column
      #Graph_Constructor.DHT_numerical(Edges_df,feature_array)
      #Edges_df['weight'] = Edges_df['weight'].apply(Graph_Constructor.DHT_numerical)
      Edges_df['weight'] = Edges_df['weight'].apply(Graph_Constructor.DHT_numerical, feature_array=feature_array)


      # Apply the function to the "weight" column to get a new DataFrame
      new_df = Edges_df.apply(Graph_Constructor.Split_Weight, axis=1)

      # Rename the columns in the new DataFrame
      new_df.columns = ['weight_'+str(i) for i in range(new_df.shape[1])]

      # Concatenate the new DataFrame with the original DataFrame
      Edges_df = pd.concat([Edges_df, new_df], axis=1)

      # Drop the original "weight" column
      Edges_df.drop('weight', axis=1, inplace=True)


      Edges_df.to_csv('D:\\PhD\\GraphImmuno_NewResults\\data\\Edges_df_values.txt', sep='\t', index=False)
      #print("The new nodes are : ", new_nodeslist)

      return New_nodes,Edges_df
      #return None



    @staticmethod
    def weight_anchor_edge(pep,hla,after_pca):
        source, target = Graph_Constructor.combinator(pep, hla)
        combine = list(itertools.product(source, target))
        weight = itertools.repeat(1, len(source) * len(target))
        edges = pd.DataFrame({'source': [item[0] for item in combine], 'target': [item[1] for item in combine], 'weight': weight})
        for i in range(edges.shape[0]):
            col1 = edges.iloc[i]['source']
            col2 = edges.iloc[i]['target']
            col3 = edges.iloc[i]['weight']
            if col1 == 'a2' or col1 == 'a9' or col1 ==  'a10':
                edges.iloc[i]['weight'] = 1.5
        feature_array = Graph_Constructor.numerical(pep, hla, after_pca)
        nodes = IndexedArray(feature_array, index=source + target)
        graph = StellarGraph(nodes, edges, node_type_default='corner', edge_type_default='line')
        return graph

    @staticmethod
    def intra_and_inter(pep,hla,after_pca):
        source, target = Graph_Constructor.combinator(pep, hla)
        combine = list(itertools.product(source, target))
        weight = itertools.repeat(2, len(source) * len(target))
        edges_inter = pd.DataFrame({'source': [item[0] for item in combine], 'target': [item[1] for item in combine], 'weight': weight})

        ## Consinderonly only linear relationships in peptides(pep and hla)
        #intra_pep = list(itertools.combinations(source,2))
        #intra_hla = list(itertools.combinations(target,2))
        intra_pep = [(source[i], source[i+1]) for i in range(len(source)-1)]
        intra_hla = [(target[i], target[i+1]) for i in range(len(target)-1)]

        intra = intra_pep + intra_hla
        #print("intra",intra)
        weight = itertools.repeat(1,len(intra))
        edges_intra = pd.DataFrame({'source':[item[0] for item in intra],'target':[item[1] for item in intra],'weight':weight})
        edges = pd.concat([edges_inter,edges_intra])
        edges = edges.set_index(pd.Index(np.arange(edges.shape[0])))

        # Updating Edge weights
        #Graph_Constructor.Update_Edges(pep,hla,edges)
        Graph_Constructor.Update_Edges_new(pep,hla,edges)

        #Removing the edges
        edges = edges[edges['weight'] != -1.00]
        Edge_Weight_List.append(edges)

        feature_array = Graph_Constructor.numerical(pep, hla, after_pca)
        nodes = IndexedArray(feature_array, index=source + target)



        # Dual Hyper Transfer
        #New_nodes,new_Edges = Graph_Constructor.DHT(source,target,edges,feature_array)
        #New_nodes,new_Edges = Graph_Constructor.DHT_new(source,target,edges,feature_array)

        # Orginal Graph
        graph = StellarGraph(nodes, edges, node_type_default='corner', edge_type_default='line')

        #new_Edges.to_csv('D:\\PhD\\New_Graph_Model\\new_Edges.txt', sep='\t', index=False)
        #print('type(new_Edges[weight]:',type(new_Edges['weight']))
        #edges.to_csv('D:\\PhD\\New_Graph_Model\\edges.txt', sep='\t', index=False)
        #print('type(edges[weight]:',type(edges['weight']))


        # Creating DHT graphs
        #graph = StellarGraph(New_nodes, new_Edges, node_type_default='corner', edge_type_default='line')
        #print("New_nodes",New_nodes)
        #print("New_nodes type : ",type(New_nodes))
        #print("new_Edges",new_Edges)
        #print("new_Edges type : ",type(new_Edges))

        #print('Nodes Type',type(nodes))
        #print('feature_array',feature_array)
        #print('feature_array size',feature_array.shape)
        #print('index', source + target)
        #print('index len', len(source + target))
        #print('Edges Type',type(edges))

        return graph

    @staticmethod
    def entrance(df,after_pca,hla_dic,dic_inventory):
        graphs = []
        graph_labels = []
        for i in range(df.shape[0]):
            print(i)
            pep = df['peptide'].iloc[i]
            try:
                hla = hla_dic[df['HLA'].iloc[i]]
            except KeyError:
                hla = hla_dic[rescue_unknown_hla(df['HLA'].iloc[i],dic_inventory)]
            label = df['immunogenicity'].iloc[i]
            #if label != 'Negative': label = 0
            #else: label = 1
            #graph = Graph_Constructor.unweight_edge(pep,hla,after_pca)
            #graph = Graph_Constructor.unweight_edge(pep,hla,after_pca)
            graph = Graph_Constructor.intra_and_inter(pep,hla,after_pca)
            graphs.append(graph)
            graph_labels.append(label)
        graph_labels = pd.Series(graph_labels)
        return graphs,graph_labels

In [ ]:
def train_fold(model, train_gen, test_gen, es, epochs):
    history = model.fit(
        train_gen, epochs=epochs, validation_data=test_gen, verbose=2, callbacks=[es],)
    # calculate performance on the test data and return along with history
    test_metrics = model.evaluate(test_gen, verbose=0)
    test_acc = test_metrics[model.metrics_names.index("acc")]
    return history, test_acc

In [ ]:
def get_generators(train_index, test_index, graph_labels, batch_size):
    train_gen = generator.flow(
        train_index, targets=graph_labels.iloc[train_index].values, batch_size=batch_size)
    test_gen = generator.flow(
        test_index, targets=graph_labels.iloc[test_index].values, batch_size=batch_size)

    return train_gen, test_gen

In [ ]:
def draw_ROC(y_true,y_pred):

    fpr,tpr,_ = roc_curve(y_true,y_pred,pos_label=1)
    area_mine = auc(fpr,tpr)
    fig = plt.figure()
    lw = 2
    plt.plot(fpr, tpr, color='darkorange',
            lw=lw, label='ROC curve (area = %0.2f)' % area_mine)
    plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver operating characteristic example')
    plt.legend(loc="lower right")
    plt.show()

In [ ]:
def draw_PR(y_true,y_pred):
    precision,recall,_ = precision_recall_curve(y_true,y_pred,pos_label=1)
    area_PR = auc(recall,precision)
    baseline = np.sum(np.array(y_true) == 1) / len(y_true)

    plt.figure()
    lw = 2
    plt.plot(recall,precision, color='darkorange',
            lw=lw, label='PR curve (area = %0.2f)' % area_PR)
    plt.plot([0, 1], [baseline, baseline], color='navy', lw=lw, linestyle='--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('Recall')
    plt.ylabel('Precision')
    plt.title('PR curve example')
    plt.legend(loc="lower right")
    plt.show()

In [ ]:
def draw_history(history):
    plt.subplot(211)
    plt.title('Loss')
    plt.plot(history.history['loss'], label='train')
    plt.plot(history.history['val_loss'], label='validation')
    plt.legend()
    # plot accuracy during training
    plt.subplot(212)
    plt.title('Accuracy')
    plt.plot(history.history['acc'], label='train')
    plt.plot(history.history['val_acc'], label='validation')
    plt.legend()
    plt.show()

In [ ]:
def hla_df_to_dic(hla):
    dic = {}
    for i in range(hla.shape[0]):
        col1 = hla['HLA'].iloc[i]  # HLA allele
        col2 = hla['pseudo'].iloc[i]  # pseudo sequence
        dic[col1] = col2
    return dic

In [ ]:
def retain_910(ori):
    cond = []
    for i in range(ori.shape[0]):
        peptide = ori['peptide'].iloc[i]
        if len(peptide) == 9 or len(peptide) == 10:
            cond.append(True)
        else:
            cond.append(False)
    data = ori.loc[cond]
    data = data.set_index(pd.Index(np.arange(data.shape[0])))
    return data

In [ ]:
    #ori_train = pd.read_csv('remove0123_sample100.csv')
    #ori_train = pd.read_csv('remove0123_sample100_Test.csv')
    #hla = pd.read_csv('hla2paratopeTable_aligned.txt',sep='\t')
    #after_pca = np.loadtxt('after_pca.txt')
    Edge_Weight_List = []

    #ori_train = pd.read_csv('D:\\PhD\Subgraph_Sampling_SLOT_GNN\\Data\\remove0123_sample100_Test_Copy2.csv')
    ori_train = pd.read_csv('D:\\PhD\Subgraph_Sampling_SLOT_GNN\\Data\\remove0123_sample100.csv')
    hla = pd.read_csv('D:\\PhD\\GraphImmuno_NewResults\\data\\hla2paratopeTable_aligned.txt',sep='\t')
    after_pca = np.loadtxt('D:\\PhD\\GraphImmuno_NewResults\\data\\after_pca.txt')

    hla_dic = hla_df_to_dic(hla)
    inventory = list(hla_dic.keys())
    dic_inventory = dict_inventory(inventory)

    ori_train['immunogenicity'], ori_train['potential'] = ori_train['potential'], ori_train['immunogenicity']

In [ ]:
graphs, graph_labels = Graph_Constructor.entrance(ori_train, after_pca, hla_dic, dic_inventory)
generator = PaddedGraphGenerator(graphs=graphs)


In [ ]:
#Sub Graph preparation
import random
import stellargraph as sg

subgraphs = []
subgraph_labels = []

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt

In [ ]:
for i in range(len(graphs)):
    # Convert the nodes to a list
    all_nodes = list(graphs[i].nodes())

    # Compute eigenvector centrality for nodes using the graph structure
    #eigenvector_centrality = nx.eigenvector_centrality_numpy(graphs[i].to_networkx(), weight=None)  # Replace 'None' with edge weights if applicable
    eigenvector_centrality = nx.eigenvector_centrality_numpy(graphs[i].to_networkx(), weight='weight')

    # Create the adjacency matrix
    adj_matrix = np.array(graphs[i].to_adjacency_matrix().toarray())

    # Set the damping factor for PageRank
    damping_factor = 0.85

    # Initialize PageRank scores
    num_nodes = len(adj_matrix)
    pagerank = np.ones(num_nodes) / num_nodes

    # Maximum number of iterations and convergence threshold for PageRank
    max_iterations = 100
    threshold = 1e-6

    # Perform PageRank iterations
    for _ in range(max_iterations):
        new_pagerank = np.zeros(num_nodes)

        for k in range(num_nodes):
            for j in range(num_nodes):
                if adj_matrix[j, k] != 0:
                    # Contribution to new PageRank from node j
                    new_pagerank[k] += damping_factor * pagerank[j] / np.sum(adj_matrix[j])

        # Handling teleportation and damping factor
        new_pagerank += (1 - damping_factor) / num_nodes

        # Check for convergence
        if np.sum(np.abs(new_pagerank - pagerank)) < threshold:
            break

        pagerank = new_pagerank


    # Create a dictionary of node names and their PageRank scores
    pagerank_scores = {all_nodes[l]: pagerank[l] for l in range(num_nodes)}

    # Combine custom centrality scores and PageRank scores
    combined_scores = {
        node: eigenvector_centrality[node] + pagerank_scores.get(node, 0.0)
        #for node in graphs[i].nodes()
        for node in all_nodes
    }

    # Find the most influential node based on the combined scores
    most_influential_node = max(combined_scores, key=combined_scores.get)
    start_node = most_influential_node

    # Print the most significant node and its degree centrality value
    print("Most Significant Node:",i, start_node)
    #print("Degree Centrality:", combined_scores)

    # Initialize the subgraph nodes with the start node
    subgraph_nodes = set([start_node])

    # Perform BFS traversal starting from the randomly chosen node
    queue = [start_node]
    max_hops = 2  # Adjust max_hops as needed

    # Perform BFS traversal starting from the randomly chosen node
    while queue and max_hops > 0:
        max_hops -= 1
        current_node = queue.pop(0)
        neighbors = graphs[i].neighbors(current_node)
        for neighbor in neighbors:
            if neighbor not in subgraph_nodes:
                subgraph_nodes.add(neighbor)
                queue.append(neighbor)


    # Create a subgraph from the selected nodes
    subgraph = graphs[i].subgraph(list(subgraph_nodes))

    # Get the edges of the subgraph
    subgraph_edges = list(subgraph.edges())

    # Initialize a list to store subgraph edge features
    subgraph_edge_features = []

    # Iterate through subgraph edges and get their features
    for edge in subgraph_edges:
        source, target = edge
        # Check if the edge exists in the main graph
        if edge in graphs[i].edges():
            # Get the index of the edge in the main graph
            edge_index = list(graphs[i].edges()).index(edge)
            # Get edge features using the index
            edge_features = list(graphs[i].edge_features())[edge_index]
            #subgraph_edge_features.append([source, target] + edge_features)
            subgraph_edge_features.append([source, target] + list(edge_features))


    # Create a DataFrame for subgraph edges with columns 'source', 'target', and edge feature values
    subgraph_edges_df = pd.DataFrame(subgraph_edge_features, columns=['source', 'target'] + [f'edge_feature_{i}' for i in range(len(edge_features))])


    # Get the index positions of subgraph nodes in the original graph
    node_positions = [all_nodes.index(node) for node in subgraph_nodes]

    # Get node features using node positions
    node_features = graphs[i].node_features()[node_positions]

    # Create a DataFrame for subgraph nodes with selected node features
    subgraph_nodes_df = pd.DataFrame(node_features, index=list(subgraph_nodes))

    subgraph = StellarGraph(subgraph_nodes_df,subgraph_edges_df,  node_type_default='corner', edge_type_default='line')

    subgraphs.append(subgraph)

#subgraph_labels = graph_labels

In [ ]:
gc_model = GCNSupervisedGraphClassification(
            layer_sizes=[64, 64],
            activations=["relu", "relu"],
            generator=generator,
            dropout=0.2, )


In [ ]:
x_inp, x_out = gc_model.in_out_tensors()
predictions = Dense(units=32, activation="relu")(x_out)
predictions = Dense(units=16, activation="relu")(predictions)
predictions = Dense(units=1, activation="sigmoid")(predictions)
model = Model(inputs=x_inp, outputs=predictions)

In [ ]:
model.compile(optimizer=Adam(0.001), loss=mean_squared_error)

In [ ]:
train_index, test_index = model_selection.train_test_split(ori_train.index, train_size=0.5)

In [ ]:
def get_generators1(train_index, test_index, graph_labels):
    train_gen = generator.flow(
        train_index, targets=graph_labels.iloc[train_index].values)
    test_gen = generator.flow(
        test_index, targets=graph_labels.iloc[test_index].values)

    return train_gen, test_gen

In [ ]:
!pip install networkx

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(subgraphs, graph_labels, test_size = 0.33, random_state = 0)

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt

In [ ]:
import pandas as pd
import numpy as np

import stellargraph as sg
from stellargraph.mapper import PaddedGraphGenerator
from stellargraph.layer import DeepGraphCNN
from stellargraph import StellarGraph

from stellargraph import datasets

from sklearn import model_selection
from IPython.display import display, HTML

from tensorflow.keras import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Dense, Conv1D, MaxPool1D, Dropout, Flatten
from tensorflow.keras.losses import binary_crossentropy
import tensorflow as tf

In [ ]:
summary = pd.DataFrame(
    [(g.number_of_nodes(), g.number_of_edges()) for g in graphs],
    columns=["nodes", "edges"],
)
summary.describe().round(1)

In [ ]:
summary = pd.DataFrame(
    [(g.number_of_nodes(), g.number_of_edges()) for g in subgraphs],
    columns=["nodes", "edges"],
)
summary.describe().round(1)

In [ ]:
graph_labels.value_counts().to_frame()

In [ ]:
graph_labels = pd.get_dummies(graph_labels, drop_first=True)

In [ ]:
ori_train1 = ori_train

In [ ]:
ori_train1.loc[ori_train1["potential"] != "Negative", "potential"] = 'Positive'
ori_train1['potential'] = ori_train1['potential'].astype('category')
ori_train1['potential'] = pd.factorize(ori_train1['potential'])[0]

In [ ]:
graph_labels = ori_train1['potential']

In [ ]:
graph_labels.value_counts().to_frame()

In [ ]:
generator = PaddedGraphGenerator(graphs=subgraphs)

Deep CGN

In [ ]:
k = 35  # the number of rows for the output tensor
layer_sizes = [32, 32, 32, 1]

dgcnn_model = DeepGraphCNN(
    layer_sizes=layer_sizes,
    activations=["tanh", "tanh", "tanh", "tanh"],
    k=k,
    bias=False,
    generator=generator,
)
x_inp, x_out = dgcnn_model.in_out_tensors()

In [ ]:
x_out = Conv1D(filters=16, kernel_size=sum(layer_sizes), strides=sum(layer_sizes))(x_out)
x_out = MaxPool1D(pool_size=2)(x_out)

x_out = Conv1D(filters=32, kernel_size=5, strides=1)(x_out)

x_out = Flatten()(x_out)

x_out = Dense(units=128, activation="relu")(x_out)
x_out = Dropout(rate=0.5)(x_out)

predictions = Dense(units=1, activation="sigmoid")(x_out)

In [ ]:
model = Model(inputs=x_inp, outputs=predictions)

model.compile(
    optimizer=Adam(lr=0.0001), loss=binary_crossentropy, metrics=["acc"],
)

In [ ]:
train_graphs, test_graphs = model_selection.train_test_split(
    graph_labels, train_size=0.9, test_size=None, stratify=graph_labels,
)

In [ ]:
gen = PaddedGraphGenerator(graphs=subgraphs)

train_gen = gen.flow(
    list(train_graphs.index - 1),
    targets=train_graphs.values,
    batch_size=50,
    symmetric_normalization=False,
)

test_gen = gen.flow(
    list(test_graphs.index - 1),
    targets=test_graphs.values,
    batch_size=1,
    symmetric_normalization=False,
)

In [ ]:
epochs = 100

In [ ]:
history = model.fit(
    train_gen, epochs=epochs, verbose=1, validation_data=test_gen, shuffle=True,
)

In [ ]:
sg.utils.plot_history(history)

In [ ]:
test_metrics = model.evaluate(test_gen)
print("\nTest Set Metrics:")
for name, val in zip(model.metrics_names, test_metrics):
    print("\t{}: {:0.4f}".format(name, val))

In [ ]:
plt.figure(figsize=(6,4))
#plt.tight_layout()
#plt.yscale('log')
plt.xlabel('Episodes')
plt.ylabel('Loss')

#create smooth line chart
plt.plot(history.history["loss"],  label='Training Loss',marker='1',linewidth=0.7)
ax = plt.gca()

plt.plot(history.history["val_loss"], label='Validation Loss',marker='.',markevery=1, linewidth=0.7)
#plt.plot(f, y1moothed , label='Actor/Critic-FSO/RF',marker='*',linewidth=0.7)
handles, labels = ax.get_legend_handles_labels()
ax.legend(handles, labels)
ax.grid(False)
#plt.fill_between(d, 1, c['Loss']+1, color='blue', alpha=.25)
%config InlineBackend.figure_format = 'svg'
#plt.savefig('/content/content1/LossvsEpisodeAll.svg')
#plt.ylim([0, 1])
plt.show()
#y_smooth = interpolate.interp1d(a['Loss'],b)
#plt.plot(make_interp_spline(b, a['Loss']), c='g', marker='.')

In [ ]:
plt.figure(figsize=(6,4))
#plt.tight_layout()
#plt.yscale('log')
plt.xlabel('Episodes')
plt.ylabel('Accuracy')

#create smooth line chart
plt.plot(history.history["acc"],  label='Training Accuracy',marker='1',linewidth=0.7)
ax = plt.gca()

plt.plot(history.history["val_acc"], label='Validation Accuracy',marker='.',markevery=1, linewidth=0.7)
#plt.plot(f, y1moothed , label='Actor/Critic-FSO/RF',marker='*',linewidth=0.7)
handles, labels = ax.get_legend_handles_labels()
ax.legend(handles, labels)
ax.grid(False)
#plt.fill_between(d, 1, c['Loss']+1, color='blue', alpha=.25)
%config InlineBackend.figure_format = 'svg'
#plt.savefig('/content/content1/LossvsEpisodeAll.svg')
#plt.ylim([0, 1])
plt.show()
#y_smooth = interpolate.interp1d(a['Loss'],b)
#plt.plot(make_interp_spline(b, a['Loss']), c='g', marker='.')

In [ ]:
graphs[1].node_features()

In [ ]:
graphs[1].edge_features()